To create a bipartite graph in Neo4j where users and movies are separated into two groups and the links are given by the ratings, you can use the following Cypher queries:
- Load the CSV files and create nodes for users and movies:

```sql
LOAD CSV WITH HEADERS FROM "file:///C:/movies.csv" AS line
MERGE (m:Movie {movieId: line.movieId, title: line.title})
```

```sql
LOAD CSV WITH HEADERS FROM "file:///C:/ratings.csv" AS line
MERGE (u:User {userId: line.userId})
```

- Create relationships between users and movies based on the ratings:

```sql
LOAD CSV WITH HEADERS FROM "file:///C:/ratings.csv" AS line
MATCH (u:User {userId: line.userId}), (m:Movie {movieId: line.movieId})
MERGE (u)-[:RATED {rating: toFloat(line.rating)}]->(m)
```

These queries will create a bipartite graph with User and Movie nodes and RATED relationships that include the rating given by the user to the movie

To project the bipartite graph onto the two different sets of points (users and movies), you can use the following Cypher queries:

- Project the bipartite graph onto the user nodes:

```sql
MATCH (u1:User)-[r1:RATED]->(m:Movie)<-[r2:RATED]-(u2:User)
WHERE u1 <> u2
MERGE (u1)-[s:SIMILAR]->(u2)
ON CREATE SET s.commonMovies = 1
ON MATCH SET s.commonMovies = s.commonMovies + 1
```
Project the bipartite graph onto the movie nodes:

```sql
MATCH (m1:Movie)<-[r1:RATED]-(u:User)-[r2:RATED]->(m2:Movie)
WHERE m1 <> m2
MERGE (m1)-[s:SIMILAR]->(m2)
ON CREATE SET s.commonUsers = 1
ON MATCH SET s.commonUsers = s.commonUsers + 1
```

These queries will create SIMILAR relationships between users and between movies based on the common movies they have rated and the common users who have rated them, respectively.

In [1]:
# Setup Connection
from neo4j import GraphDatabase

uri = "neo4j://localhost:7687"
username = "neo4j"
password = "testtest"
driver = GraphDatabase.driver(uri, auth=(username, password))

ModuleNotFoundError: No module named 'neo4j'

in Cypher

```sql
CREATE (n:Person {name: "Alice"})
```

creates a node ( ) with label _Person_ with the property with key _name_ of value _Alice_

In [2]:
def add_node(tx, name):
    tx.run("CREATE (n:Person {name: $name})", name=name)

with driver.session() as session:
    session.execute_write(add_node, "Alice")

in Cypher

```sql
MATCH (n:Person {name: "Alice"}) RETURN n.name AS name
```

In [ ]:
def get_person(tx, name):
    result = tx.run("MATCH (n:Person {name: $name}) RETURN n.name AS name", name=name)
    return [record["name"] for record in result]

with driver.session() as session:
    names = session.execute_read(get_person, "Alice")
    print(names)

in Cypher

```sql
MATCH ()-[r]->() DELETE r
MATCH (n) DELETE n
```

In [4]:
def delete_all(tx):
    tx.run("MATCH ()-[r]->() DELETE r")
    tx.run("MATCH (n) DELETE n")
        
with driver.session() as session:
    session.execute_write(delete_all)

### Le nostre query per controllare che il la user_movie_matrix sia stata correttamente caricata

```sql
MATCH (n:User {userId: 1}) RETURN n.userId AS name 
-- Expected result: 1
```

```sql
MATCH (m:Movie  {movieId: 1}) RETURN m.title AS name 
-- Expected result: Toy Story (1995)
```

```sql
MATCH (u:User {userId: 1})-[r:RATED]->(m:Movie) RETURN u.userId, m.title, r.rating LIMIT 10 
/*
╒════════╤═════════════════════════════╤════════╕
│u.userId│m.title                      │r.rating│
╞════════╪═════════════════════════════╪════════╡
│1       │"Toy Story (1995)"           │4.0     │
├────────┼─────────────────────────────┼────────┤
│1       │"Grumpier Old Men (1995)"    │4.0     │
├────────┼─────────────────────────────┼────────┤
│1       │"Heat (1995)"                │4.0     │
├────────┼─────────────────────────────┼────────┤
│1       │"Seven (a.k.a. Se7en) (1995)"│5.0     │
├────────┼─────────────────────────────┼────────┤
│1       │"Usual Suspects, The (1995)" │5.0     │
├────────┼─────────────────────────────┼────────┤
│1       │"From Dusk Till Dawn (1996)" │3.0     │
├────────┼─────────────────────────────┼────────┤
│1       │"Bottle Rocket (1996)"       │5.0     │
├────────┼─────────────────────────────┼────────┤
│1       │"Braveheart (1995)"          │4.0     │
├────────┼─────────────────────────────┼────────┤
│1       │"Rob Roy (1995)"             │5.0     │
├────────┼─────────────────────────────┼────────┤
│1       │"Canadian Bacon (1995)"      │5.0     │
└────────┴─────────────────────────────┴────────┘
*/
```

```sql
MATCH (u:User {userId: 1})-[r:RATED]->(m:Movie) RETURN u.userId, m.title, r.rating ORDER BY r.rating
/*
╒════════╤══════════════════════════════════════════════════════════════════════╤════════╕
│u.userId│m.title                                                               │r.rating│
╞════════╪══════════════════════════════════════════════════════════════════════╪════════╡
│1       │"Talented Mr. Ripley, The (1999)"                                     │1.0     │
├────────┼──────────────────────────────────────────────────────────────────────┼────────┤
│1       │"Psycho (1960)"                                                       │2.0     │
├────────┼──────────────────────────────────────────────────────────────────────┼────────┤
│1       │"Toys (1992)"                                                         │2.0     │
├────────┼──────────────────────────────────────────────────────────────────────┼────────┤
│1       │"I Still Know What You Did Last Summer (1998)"                        │2.0     │
├────────┼──────────────────────────────────────────────────────────────────────┼────────┤
│1       │"Psycho (1998)"                                                       │2.0     │
├────────┼──────────────────────────────────────────────────────────────────────┼────────┤
│1       │"Mummy, The (1999)"                                                   │2.0     │
├────────┼──────────────────────────────────────────────────────────────────────┼────────┤
│1       │"From Dusk Till Dawn (1996)"                                          │3.0     │
├────────┼──────────────────────────────────────────────────────────────────────┼────────┤
│1       │"Clerks (1994)"                                                       │3.0     │
├────────┼──────────────────────────────────────────────────────────────────────┼────────┤
│1       │"Pulp Fiction (1994)"                                                 │3.0     │
├────────┼──────────────────────────────────────────────────────────────────────┼────────┤
...
│1       │"X-Men (2000)"                                                        │5.0     │
├────────┼──────────────────────────────────────────────────────────────────────┼────────┤
│1       │"M*A*S*H (a.k.a. MASH) (1970)"                                        │5.0     │
└────────┴──────────────────────────────────────────────────────────────────────┴────────┘
*/
```

In [3]:
userId = 1

def get_edges_for_user(tx, userId):
    # MATCH (u:User {userId: 1})-[r:RATED]->(m:Movie) RETURN u.userId AS userId, m.title AS title, r.rating AS rating ORDER BY rating
    # result = tx.run("MATCH (u:User {userId: $userId})-[r:RATED]->(m:Movie) RETURN n.userId AS userId, m.title AS title, r.rating AS rating", userId=userId)
    result = tx.run("MATCH (u:User {userId: $userId})-[r:RATED]->(m:Movie) RETURN u.userId AS userId, m.title AS title, r.rating AS rating ORDER BY rating", userId=userId)
    return {record["title"]: record["rating"] for record in result}

with driver.session() as session:
    edges = session.execute_read(get_edges_for_user, userId)
    for key, value in edges.items():
        print(key, value)

Talented Mr. Ripley, The (1999) 1.0
Psycho (1960) 2.0
Toys (1992) 2.0
I Still Know What You Did Last Summer (1998) 2.0
Psycho (1998) 2.0
Mummy, The (1999) 2.0
From Dusk Till Dawn (1996) 3.0
Clerks (1994) 3.0
Pulp Fiction (1994) 3.0
Stargate (1994) 3.0
Blown Away (1994) 3.0
Mrs. Doubtfire (1993) 3.0
Mission: Impossible (1996) 3.0
Space Jam (1996) 3.0
Twister (1996) 3.0
Independence Day (a.k.a. ID4) (1996) 3.0
Escape to Witch Mountain (1975) 3.0
Pete's Dragon (1977) 3.0
Shining, The (1980) 3.0
Batman Returns (1992) 3.0
Sneakers (1992) 3.0
Last of the Mohicans, The (1992) 3.0
McHale's Navy (1997) 3.0
Men in Black (a.k.a. MIB) (1997) 3.0
I Know What You Did Last Summer (1997) 3.0
Starship Troopers (1997) 3.0
Return to Oz (1985) 3.0
Young Sherlock Holmes (1985) 3.0
Logan's Run (1976) 3.0
Rocky Horror Picture Show, The (1975) 3.0
Encino Man (1992) 3.0
Sister Act (1992) 3.0
Toy Story (1995) 4.0
Grumpier Old Men (1995) 4.0
Heat (1995) 4.0
Braveheart (1995) 4.0
Ed Wood (1994) 4.0
Clear and Pres

```sql
MATCH (u:User {userId: 1})
RETURN COUNT {(u:User {userId: 1})-[r:RATED]->(m:Movie)} as hasRatedMovies
/*
╒══════════════╕
│hasRatedMovies│
╞══════════════╡
│232           │
└──────────────┘
*/
```

Query per controllare che il test sia stato caricato correttamente


```sql
MATCH (u:User {userId: 10})-[r:RECOMMENDED]->(m:Movie) RETURN u.userId AS user, m.title AS title
/*
╒════╤══════════════════════════════════════════════════════════════════════╕
│user│title                                                                 │
╞════╪══════════════════════════════════════════════════════════════════════╡
│10  │"Toy Story (1995)"                                                    │
├────┼──────────────────────────────────────────────────────────────────────┤
│10  │"Star Wars: Episode IV - A New Hope (1977)"                           │
├────┼──────────────────────────────────────────────────────────────────────┤
│10  │"Jurassic Park (1993)"                                                │
├────┼──────────────────────────────────────────────────────────────────────┤
│10  │"Silence of the Lambs, The (1991)"                                    │
├────┼──────────────────────────────────────────────────────────────────────┤
│10  │"Star Wars: Episode V - The Empire Strikes Back (1980)"               │
├────┼──────────────────────────────────────────────────────────────────────┤
│10  │"Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ar│
│    │k) (1981)"                                                            │
├────┼──────────────────────────────────────────────────────────────────────┤
│10  │"Star Wars: Episode VI - Return of the Jedi (1983)"                   │
├────┼──────────────────────────────────────────────────────────────────────┤
│10  │"Back to the Future (1985)"                                           │
├────┼──────────────────────────────────────────────────────────────────────┤
│10  │"Shawshank Redemption, The (1994)"                                    │
├────┼──────────────────────────────────────────────────────────────────────┤
│10  │"Terminator 2: Judgment Day (1991)"                                   │
└────┴──────────────────────────────────────────────────────────────────────┘
*/
```

Deve matchare l'output dello script

- Star Wars: Episode IV - A New Hope (1977)
- Shawshank Redemption, The (1994)
- Star Wars: Episode V - The Empire Strikes Back (1980)
- Silence of the Lambs, The (1991)
- Jurassic Park (1993)
- Star Wars: Episode VI - Return of the Jedi (1983)
- Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
- Back to the Future (1985)
- Toy Story (1995)
- Terminator 2: Judgment Day (1991)

Per la visualizzazione a grafo

```sql
MATCH p=(u:User {userdId: 1}) -[r:RECOMMENDED]->(m:Movie) RETURN p
```



```sql
MERGE z=(a:User {userdId: 1}) -[b:RECOMMENDED]->(c:Movie)
MERGE y=(d:User {userdId: 1}) -[e:RECOMMENDED]->(f:Movie)
MERGE x=(g:User {userdId: 1}) -[h:RECOMMENDED]->(i:Movie)
MERGE w=(j:User {userdId: 1}) -[k:RECOMMENDED]->(l:Movie)
RETURN z, y, x, w
```


In [4]:
driver.close()
exit(0)

: 